In [30]:
# Import dependencies
import pandas as pd
import requests
import os
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [31]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\mv021\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


## NASA Mars News
- Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [32]:
#url to page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [33]:
# Retrieve the latest news title
news_title = soup.find_all('div', class_='content_title')[0].text

# Retrieve the latest news paragraph
news_article = soup.find_all('div', class_='article_teaser_body')[0].text

print(f"News_title = {news_title}")
print("---------------------------------------------------------------------------------------")
print(f"News_paragraph = {news_article}")    

News_title = MAVEN Maps Electric Currents around Mars that are Fundamental to Atmospheric Loss
---------------------------------------------------------------------------------------
News_paragraph = Five years after NASA’s MAVEN spacecraft entered into orbit around Mars, data from the mission has led to the creation of a map of electric current systems in the Martian atmosphere.


## JPL Mars Space Images - Featured Image
- Visit the url for the Featured Space Image site here.
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Make sure to find the image url to the full size .jpg image.
-Make sure to save a complete url string for this image.

In [34]:
url_jpl = 'https://spaceimages-mars.com/'
browser.visit(url_jpl)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [35]:
#Create variable to find the image and the variable for the url
image_path = soup.find_all('img', class_='headerimage fade-in')[0]['src']
featured_image_url = url_jpl + image_path

print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


# Mars Facts
- Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
- Use Pandas to convert the data to a HTML table string.

In [36]:
url_mars = 'https://galaxyfacts-mars.com/'

#Use Pandas to read the HTML table
tables = pd.read_html(url_mars)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [37]:
# Convert list to a pandas dataframe for table 1
mars_df = tables[0]
mars_df.columns = ['Mars - Earth Comparison', 'Mars', 'Earth']
#Drop first rown and set index
mars_df = mars_df.iloc[1:]
mars_df.set_index('Mars - Earth Comparison', inplace=True)

mars_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [41]:
# Convert list to a pandas dataframe for table 2
mars2_df = tables[1]
mars2_df.columns = ['Mars Planet Profile', 'Value']
#Drop first rown and set index
mars2_df = mars2_df.iloc[1:]
mars2_df.set_index('Mars Planet Profile', inplace=True)

mars2_df

,Value
Mars Planet Profile,
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [42]:
#Convert table to an HTML table string
mars_html_table = mars2_df.to_html()

mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Mars Planet Profile</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [43]:
#Strip unwanted lines
mars_html_table = mars_html_table.replace('\n', '')

print(mars_html_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Mars Planet Profile</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


In [44]:
#Save html table to file
mars_df.to_html('table.html')

## Mars Hemispheres
- Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
- You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
- Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [27]:
# Scrape mars hemisphere title and image
url = 'https://marshemispheres.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [28]:
# Extract elements
mars_hemi = soup.find('div', class_='collapsible results')
mars_items = mars_hemi.find_all('div', class_='item')

hemi_image_urls = []

#iterate through hemispheres
for i in mars_items:
    #collect title
    hemisphere = i.find('div', class_='description')
    title = hemisphere.h3.text
    
    #collect image
    hemi_url = hemisphere.a['href']
    browser.visit(url + hemi_url)
    
    img_html = browser.html
    img_soup = BeautifulSoup(img_html, 'html.parser')
    
    img_link = img_soup.find('div', class_='downloads')
    img_url = img_link.find('li').a['href']
    img_full_url = url + img_url
    
    # Create Dictionaries
    img_dict = {}
    img_dict['Title'] = title
    img_dict['img_url'] = img_full_url
    
    hemi_image_urls.append(img_dict)
    
print(hemi_image_urls)


[{'Title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'Title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'Title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'Title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [29]:
# Create dictionary
mars_dict = {
    'news_title': news_title,
    'news_paragraph': news_article,
    'featured_image': featured_image_url,
    'mars_table': str(mars_html_table),
    'hemisphere_image': hemi_image_urls}

print(mars_dict)

{'news_title': 'MOXIE Could Help Future Rockets Launch Off Mars', 'news_paragraph': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.", 'featured_image': 'https://spaceimages-mars.com/image/featured/mars3.jpg', 'mars_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Mars Planet Profile</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>    

In [ ]:
browser.quit()